In [1]:
import os
os.chdir('../')
os.getcwd()

'/work'

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

# mushroom

## 準備

In [3]:
from process.preprocess.get_dataset import get_dataset
from process.mainprocess.split_data import split_data

In [4]:
options_classifier = {
    "experiments": {
        "n_targets": 25,
        "source_sampling": False
    },
    
    "datasets": {
        "source":"uci",
        "dataset_name": "mushroom",
        "task": "classification",
        "split_feature_name": "stalk_shape",
        "y_feature": "Attributes",
        "split_feature": "stalk_shape",
        "target_domain": None,
        "metric": "auc"
    }
}

target_domain = "t"
seed = 1

In [5]:
df = get_dataset(options_classifier["datasets"])
df

,Attributes,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,p,5.0,2.0,4.0,1.0,6.0,1.0,0.0,1.0,4.0,...,2.0,7.0,7.0,0.0,2.0,1.0,4.0,2.0,3.0,5.0
1,e,5.0,2.0,9.0,1.0,0.0,1.0,0.0,0.0,4.0,...,2.0,7.0,7.0,0.0,2.0,1.0,4.0,3.0,2.0,1.0
2,e,0.0,2.0,8.0,1.0,3.0,1.0,0.0,0.0,5.0,...,2.0,7.0,7.0,0.0,2.0,1.0,4.0,3.0,2.0,3.0
3,p,5.0,3.0,8.0,1.0,6.0,1.0,0.0,1.0,5.0,...,2.0,7.0,7.0,0.0,2.0,1.0,4.0,2.0,3.0,5.0
4,e,5.0,2.0,3.0,0.0,5.0,1.0,1.0,0.0,4.0,...,2.0,7.0,7.0,0.0,2.0,1.0,0.0,3.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,e,3.0,2.0,4.0,0.0,5.0,0.0,0.0,0.0,11.0,...,2.0,5.0,5.0,0.0,1.0,1.0,4.0,0.0,1.0,2.0
8120,e,5.0,2.0,4.0,0.0,5.0,0.0,0.0,0.0,11.0,...,2.0,5.0,5.0,0.0,0.0,1.0,4.0,0.0,4.0,2.0
8121,e,2.0,2.0,4.0,0.0,5.0,0.0,0.0,0.0,5.0,...,2.0,5.0,5.0,0.0,1.0,1.0,4.0,0.0,1.0,2.0
8122,p,3.0,3.0,4.0,0.0,8.0,1.0,0.0,1.0,0.0,...,1.0,7.0,7.0,0.0,2.0,1.0,0.0,7.0,4.0,2.0


In [6]:
X_train, y_train, X_train_target, y_train_target, X_test, y_test, domain_col = \
    split_data(options_classifier, df, target_domain, 1)

In [7]:
a = y_test

In [8]:
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)
domain_encoder = LabelEncoder()
domain_col = domain_encoder.fit_transform(domain_col)
target_domain = domain_encoder.transform(np.array([target_domain]))[0]

# X_train = pd.DataFrame(np.concatenate([X_train,domain_col.reshape(-1,1)],axis=1))
X_train = pd.DataFrame(X_train)
y_train = pd.Series(y_train)
domain_col = pd.Series(domain_col)

In [9]:
label_encoder.classes_
# np.searchsorted(label_encoder.classes_, a).reshape(-1,1).shape

array(['e', 'p'], dtype=object)

In [10]:
X_train.shape, X_test.shape

((3541, 21), (4583, 21))

In [11]:
# X_train_target = pd.DataFrame(np.concatenate([X_train_target, domain_col[-X_train_target.shape[0]:].reshape(-1,1)],axis=1))
# y_train_target = pd.Series(y_train_target)

# X_train_source = pd.DataFrame(np.concatenate([X_train[domain_col!=target_domain],domain_col[:X_train_source.shape[0]].reshape(-1,1)],axis=1))
# y_train_source = pd.Series(y_train[domain_col!=target_domain])

## trbagg実行

In [12]:
from tl_algs import trbag
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score

In [13]:
model = trbag.TrBag(test_set_X=X_test, 
                  test_set_domain=target_domain, 
                  train_pool_X=X_train, 
                  train_pool_y=y_train, 
                  train_pool_domain=domain_col, 
                  sample_size=y_train.shape[0],
                  Base_Classifier=DecisionTreeClassifier,
                  classifier_params={'max_depth':6},
                  T=100,
                  rand_seed=seed
                 )

In [14]:
confidences, predictions = model.train_filter_test()

In [15]:
np.vstack([confidences, 1-confidences]).T.shape

(4583, 2)

In [16]:
predictions.shape ,y_test.shape

((4583,), (4583,))

In [17]:
roc_auc_score(y_test, predictions)

0.9813736903376018

In [18]:
roc_auc_score(y_test, confidences)

0.976222380588154

In [21]:
# filter_funcにMVVを用いた場合(今回はtarget domainのデータ数が少ないため△)

model2 = trbag.TrBag(test_set_X=X_test, 
                  test_set_domain=target_domain, 
                  train_pool_X=X_train, 
                  train_pool_y=y_train, 
                  train_pool_domain=domain_col, 
                  sample_size=y_train.shape[0],
                  Base_Classifier=DecisionTreeClassifier,
                  classifier_params={'max_depth':6},
                  filter_func=trbag.mvv_filter,
                  T=100,
                  rand_seed=seed
                 )
confidences2, predictions2 = model.train_filter_test()

roc_auc_score(y_test, confidences2)

0.976222380588154

In [61]:
target_domains = ["e","t"]
exp_iters=30
results_classifier = np.zeros((exp_iters,len(target_domains)))

df = get_dataset(options_classifier["datasets"])

for i in range(exp_iters):
    for j, target_domain in enumerate(target_domains):
        X_train, y_train, X_train_target, y_train_target, X_test, y_test, domain_col = \
            split_data(options_classifier, df, target_domain, i)
        label_encoder = LabelEncoder()
        y_train = label_encoder.fit_transform(y_train)
        y_test = label_encoder.transform(y_test)
        domain_encoder = LabelEncoder()
        domain_col = domain_encoder.fit_transform(domain_col)
        target_domain = domain_encoder.transform(np.array([target_domain]))[0]
        X_train = pd.DataFrame(X_train)
        y_train = pd.Series(y_train)
        domain_col = pd.Series(domain_col)
        model = trbag.TrBag(test_set_X=X_test, 
                          test_set_domain=target_domain, 
                          train_pool_X=X_train, 
                          train_pool_y=y_train, 
                          train_pool_domain=domain_col, 
                          sample_size=y_train.shape[0],
                          Base_Classifier=DecisionTreeClassifier,
                          classifier_params={'max_depth':6},
                          T=100,
                          rand_seed=seed
                         )
        _, predictions = model.train_filter_test()
        results_classifier[i,j] = (roc_auc_score(y_test, predictions))**0.5

In [62]:
results_classifier

array([[0.9423935 , 0.99064307],
       [0.96700505, 0.99064307],
       [0.95736146, 0.99064672],
       [0.94695998, 0.99064672],
       [0.96090888, 0.98008546],
       [0.97691499, 0.99063942],
       [0.90491598, 0.99299875],
       [0.91161547, 0.99065764],
       [0.94320941, 0.99299602],
       [0.9594361 , 0.99065036],
       [0.92158041, 0.99063577],
       [0.93334033, 0.99064672],
       [0.91887209, 0.9953289 ],
       [0.9734425 , 0.99377184],
       [0.94219497, 0.99533797],
       [0.92894777, 0.9213274 ],
       [0.95048728, 0.990654  ],
       [0.92508184, 0.99065036],
       [0.8804817 , 0.99698575],
       [0.94623969, 0.99064307],
       [0.86289164, 0.99407157],
       [0.90635224, 0.96619557],
       [0.94105351, 0.99063942],
       [0.96810042, 0.99066127],
       [0.92512904, 0.99065036],
       [0.94203451, 0.99064672],
       [0.95816393, 0.99299875],
       [0.89469382, 0.99249904],
       [0.94641139, 0.99533434],
       [0.96371457, 0.99447   ]])

In [63]:
results_classifier.mean(axis=0)

array([0.93666448, 0.98849187])

# concrete

## 準備

In [64]:
options_regressor = {
    "experiments": {
        "n_targets": 25,
        "source_sampling": False
    },
    
    "datasets": {
        "source":"uci",
        "dataset_name": "concrete",
        "task": "regression",
        "split_feature_name": "Water",
        "y_feature": "Concrete compressive strength",
        "split_feature": "Water_CAT",
        "n_domains": 3,
        "target_domain": None,
        "metric": "RMSE"
    }
}

target_domain = 1
seed = 1

In [65]:
df = get_dataset(options_regressor["datasets"])
df

,Cement,Blast Furnace Slag,Fly Ash,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Concrete compressive strength,Water_CAT
0,540.0,0.0,0.0,2.5,1055.0,676.0,28,61.887366,0
1,332.5,142.5,0.0,0.0,932.0,594.0,270,40.269535,2
2,332.5,142.5,0.0,0.0,932.0,594.0,365,41.052780,2
3,198.6,132.4,0.0,0.0,978.4,825.5,360,44.296075,1
4,266.0,114.0,0.0,0.0,932.0,670.0,90,47.029847,2
...,...,...,...,...,...,...,...,...,...
1024,276.4,116.0,90.3,8.9,870.1,768.3,28,44.284354,1
1025,322.2,0.0,115.6,10.4,817.9,813.4,28,31.178794,2
1026,148.5,139.4,108.6,6.1,892.4,780.0,28,23.696601,2
1027,159.1,186.7,0.0,11.3,989.6,788.9,28,32.768036,1


In [66]:
X_train, y_train, X_train_target, y_train_target, X_test, y_test, domain_col = \
    split_data(options_regressor, df, target_domain, 1)

X_train = pd.DataFrame(X_train)
y_train = pd.Series(y_train)
domain_col = pd.Series(domain_col)

## Trbagg実行

In [67]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from tl_algs import voter

In [68]:
model = trbag.TrBag(test_set_X=X_test, 
                  test_set_domain=target_domain, 
                  train_pool_X=X_train, 
                  train_pool_y=y_train, 
                  train_pool_domain=domain_col, 
                  sample_size=y_train.shape[0],
                  Base_Classifier=DecisionTreeRegressor,
                  filter_metric=mean_squared_error,
                  T=100,
                  vote_func=voter.count_vote_regression,
                  rand_seed=seed
                 )

In [71]:
model.train_filter_test().shape, y_test.shape

((402,), (402,))

In [75]:
predictions = model.train_filter_test()
predictions

array([33.19206412, 32.922479  , 33.00383717, 33.19206412, 37.6798634 ,
       33.19206412, 13.22414968, 10.3938507 , 10.7875415 ,  7.83934212,
       12.54156844, 37.6798634 , 25.11662572, 36.3009114 , 25.2003478 ,
       25.2003478 , 32.82319446, 46.20178676, 35.07640202, 46.80163088,
       34.67374804, 53.57917996, 60.2946762 , 55.49592324, 71.29871316,
       57.21271848, 53.57917996, 64.30053176, 63.3973182 , 77.29715436,
       56.8128224 , 53.65502232, 65.19685056, 79.29663476, 56.8128224 ,
       10.0318758 , 22.13907436, 22.83544512, 27.66177712, 34.55653712,
       13.35515012, 21.50475644, 24.53845084, 32.8535314 , 12.5484632 ,
       19.41564416, 24.24197616, 32.8535314 , 39.2311844 , 11.98309288,
       31.2677366 , 31.64005364, 43.49904084, 48.67011084, 11.98309288,
       31.2677366 , 31.64005364, 43.49904084, 48.67011084, 10.38350856,
       22.31833812, 25.72434956, 33.19206412, 34.55653712, 10.38350856,
       22.31833812, 28.62704352, 27.66177712, 34.55653712, 13.62

In [73]:
(predictions, predictions)

(array([False, False, False, ...,  True,  True,  True]),
 array([False, False, False, ...,  True,  True,  True]))

In [79]:
(mean_squared_error(y_test, predictions))**0.5

6.038023184626992

In [86]:
target_domains = [0,1,2]
exp_iters=30
results = np.zeros((exp_iters,len(target_domains)))

df = get_dataset(options_regressor["datasets"])

for i in range(exp_iters):
    for j, target_domain in enumerate(target_domains):
        X_train, y_train, X_train_target, y_train_target, X_test, y_test, domain_col = \
            split_data(options_regressor, df, target_domain, i)
        X_train = pd.DataFrame(X_train)
        y_train = pd.Series(y_train)
        domain_col = pd.Series(domain_col)
        model = trbag.TrBag(test_set_X=X_test, 
                          test_set_domain=target_domain, 
                          train_pool_X=X_train, 
                          train_pool_y=y_train, 
                          train_pool_domain=domain_col, 
                          sample_size=y_train.shape[0],
                          Base_Classifier=DecisionTreeRegressor,
                          filter_metric=mean_squared_error,
                          classifier_params={'max_depth':6},
                          T=100,
                          vote_func=voter.count_vote_regression,
                          rand_seed=seed
                         )
        predictions = model.train_filter_test()
        results[i,j] = (mean_squared_error(y_test, predictions))**0.5

In [87]:
results

array([[10.69478804,  7.12178097,  8.31276361],
       [10.42479688,  7.1409663 ,  7.80689953],
       [10.26660714,  7.18886203,  7.31417211],
       [ 9.85357994,  7.1996941 ,  7.52584743],
       [ 9.98530217,  7.08977725,  7.63861926],
       [10.20631305,  7.16123259,  8.02348199],
       [10.75642784,  7.71636771,  7.57716796],
       [ 9.79720285,  7.4203326 ,  7.91227642],
       [10.06403128,  7.00995426,  7.59942731],
       [ 9.84311701,  7.63921966,  7.59458122],
       [ 9.31924345,  7.49428013,  8.48031942],
       [ 9.90552393,  7.35485772,  8.04120767],
       [ 9.48495198,  7.46042466,  7.38782545],
       [10.07767409,  7.06918669,  7.60819901],
       [ 9.11619543,  6.91284575,  7.51879291],
       [10.38857362,  7.21340293,  8.01647777],
       [10.12733788,  6.80411048,  7.60084147],
       [10.57255317,  7.48050226,  8.12713511],
       [ 9.53285693,  7.33120508,  8.11097507],
       [10.5363209 ,  7.45865204,  8.1343674 ],
       [ 9.98135309,  6.97093003,  7.373

In [88]:
results.mean(axis=0)

array([10.06325437,  7.19442361,  7.80833251])

# banknote

In [106]:
options_banknote = {
    "experiments": {
        "n_targets": 25,
        "source_sampling": False
    },
    
    "datasets": {
        "source":"uci",
        "dataset_name": "banknote",
        "task": "classification",
        "split_feature_name": "curtosis",
        "y_feature": "class (integer)",
        "split_feature": "curtosis_CAT",
        "target_domain": None,
        "metric": "auc"
    }
}

In [108]:
target_domains = [0,1,2]
exp_iters=30
results_banknote = np.zeros((exp_iters,len(target_domains)))

df = get_dataset(options_banknote["datasets"])

for i in range(exp_iters):
    for j, target_domain in enumerate(target_domains):
        X_train, y_train, X_train_target, y_train_target, X_test, y_test, domain_col = \
            split_data(options_banknote, df, target_domain, i)
        label_encoder = LabelEncoder()
        y_train = label_encoder.fit_transform(y_train)
        y_test = label_encoder.transform(y_test)
        domain_encoder = LabelEncoder()
        domain_col = domain_encoder.fit_transform(domain_col)
        target_domain = domain_encoder.transform(np.array([target_domain]))[0]
        X_train = pd.DataFrame(X_train)
        y_train = pd.Series(y_train)
        domain_col = pd.Series(domain_col)
        model = trbag.TrBag(test_set_X=X_test, 
                          test_set_domain=target_domain, 
                          train_pool_X=X_train, 
                          train_pool_y=y_train, 
                          train_pool_domain=domain_col, 
                          sample_size=y_train.shape[0],
                          Base_Classifier=DecisionTreeClassifier,
                          classifier_params={'max_depth':6},
                          T=100,
                          rand_seed=seed
                         )
        _, predictions = model.train_filter_test()
        results_banknote[i,j] = (roc_auc_score(y_test, predictions))**0.5

In [109]:
results_banknote

array([[0.86171619, 0.92156111, 0.92599463],
       [0.8889295 , 0.96575075, 0.92616293],
       [0.85546113, 0.95780283, 0.92461382],
       [0.88231338, 0.91243921, 0.92024282],
       [0.90169361, 0.96998517, 0.93094934],
       [0.87028298, 0.96618519, 0.93981908],
       [0.89943775, 0.96642358, 0.93571126],
       [0.92328707, 0.91860357, 0.91207331],
       [0.89131047, 0.96820759, 0.92547406],
       [0.82985069, 0.9474765 , 0.90670305],
       [0.8876237 , 0.97354326, 0.91772666],
       [0.8676673 , 0.96990666, 0.93836612],
       [0.88433222, 0.9622832 , 0.93718653],
       [0.88710899, 0.95131732, 0.92304818],
       [0.88102763, 0.96594962, 0.91154697],
       [0.89045665, 0.96700113, 0.92255683],
       [0.9015148 , 0.96822008, 0.93133845],
       [0.86684908, 0.9672568 , 0.93689795],
       [0.87182604, 0.91788885, 0.91656429],
       [0.86324985, 0.94352255, 0.91656429],
       [0.877423  , 0.91824688, 0.91855865],
       [0.88231338, 0.92937344, 0.93801708],
       [0.

In [110]:
results_banknote.mean(axis=0)

array([0.88201728, 0.9531202 , 0.92638042])

# qsar

In [111]:
options_qsar = {
    "experiments": {
        "n_targets": 25,
        "source_sampling": False
    },
    
    "datasets": {
        "source":"uci",
        "dataset_name": "qsar",
        "task": "regression",
        "split_feature_name": "SM1_Dz(Z)",
        "y_feature": "quantitative response",
        "split_feature": "SM1_Dz_CAT",
        "target_domain": None,
        "metric": "RMSE"
    }
}

In [117]:
target_domains = [0,1,2]
exp_iters=30
results_qsar = np.zeros((exp_iters,len(target_domains)))

df = get_dataset(options_qsar["datasets"])

for i in range(exp_iters):
    for j, target_domain in enumerate(target_domains):
        X_train, y_train, X_train_target, y_train_target, X_test, y_test, domain_col = \
            split_data(options_qsar, df, target_domain, i)
        X_train = pd.DataFrame(X_train)
        y_train = pd.Series(y_train)
        domain_col = pd.Series(domain_col)
        model = trbag.TrBag(test_set_X=X_test, 
                          test_set_domain=target_domain, 
                          train_pool_X=X_train, 
                          train_pool_y=y_train, 
                          train_pool_domain=domain_col, 
                          sample_size=y_train.shape[0],
                          Base_Classifier=DecisionTreeRegressor,
                          filter_metric=mean_squared_error,
                          classifier_params={'max_depth':6},
                          T=100,
                          vote_func=voter.count_vote_regression,
                          rand_seed=seed
                         )
        predictions = model.train_filter_test()
        results_qsar[i,j] = (mean_squared_error(y_test, predictions))**0.5

In [118]:
results_qsar

array([[0.99256612, 1.07302463, 1.41527062],
       [0.97596928, 1.08535913, 1.37646301],
       [0.9631967 , 1.06229377, 1.40910112],
       [0.99061115, 1.09665362, 1.43015467],
       [0.95994034, 1.03760779, 1.38485881],
       [0.94101819, 1.08105474, 1.38650922],
       [0.96980988, 1.08098467, 1.40480649],
       [0.97114222, 1.09116061, 1.36431362],
       [0.96601605, 1.05556614, 1.33543087],
       [0.99995109, 1.0540139 , 1.44600951],
       [0.96312743, 1.0694366 , 1.4289287 ],
       [0.95599731, 1.04729185, 1.43699333],
       [0.99435146, 1.05317576, 1.43492419],
       [0.99873592, 1.01909555, 1.40016371],
       [0.94447594, 0.99331331, 1.4074073 ],
       [0.98396129, 1.09506945, 1.41872938],
       [0.986575  , 1.077641  , 1.39488167],
       [0.98089169, 1.08205107, 1.42927008],
       [0.9962442 , 1.02064618, 1.34504431],
       [0.9627645 , 1.06837131, 1.40503808],
       [0.9424278 , 1.03625425, 1.41379602],
       [1.00232422, 1.08254595, 1.41422446],
       [0.

In [119]:
results_qsar.mean(axis=0)

array([0.97114967, 1.06184988, 1.40301321])